In [1]:
import pandas as pd

In [122]:
filename_query_map = {
    'fraud_100_penalty.txt': 'penalty or consequences for the fraudster, or for others',
    'fraud_100_nature.txt': 'nature of the fraud: the amount of money taken, benefits for the fraudster, and how the fraud worked',
    'eq_100_geo.txt': 'geography, region, or location',
    'eq_100_rescue.txt': 'recovery and aid efforts (death toll and injuries, foreign/domestic government assistance, impact on survivors)',
 

}

filename_aspect_map: {
    'fraud_100_penalty.txt': 'penalty of fraud',
    'fraud_100_nature.txt': 'nature of fraid',
    'eq_100_geo.txt': 'earthquake geography',
    'eq_100_rescue': 'earthquake_recovery'
}

def read_annotations(filepath, filename):
    with open(f'{filepath}/{filename}', 'r') as fp:
        all_lines = fp.readlines()
        annotations = ''.join(all_lines).split('\n\n\n')
    query =  filename_query_map[filename]
    aspect = filename_aspect_map[filename]
    return annotations, query, aspect

def get_article_summaries(annotations):
    articles = []
    reference_summaries = []
    for example in annotations:
        article = []
        all_sentences = []
        for sentence in example.split('\n'):
            if 'Annotator' in sentence:
                annotations = sentence.split('Sentences')[1].split(',')
                sentences_marked = []
                for each_ann in annotations:
                    sentence_num = eval(re.findall('^.*?(\d+)', each_ann)[0])
                    sentences_marked.append(sentence_num)
                all_sentences += sentences_marked 

            else:
                sentence = ' '.join(sentence.split(':')[1:]).strip()
                article.append(sentence)
        reference_summ_idx = set([each for each in all_sentences if all_sentences.count(each) >= 2])
        # print(reference_summ_idx)
        reference_summ = [article[idx] for idx in reference_summ_idx]

        articles += ['\n'.join(article)]
        reference_summaries += ['\n'.join(reference_summ)]
    return articles, reference_summaries
    



In [123]:
import random

def populate_df(filepath, filename, df_dict):
    annotations, query, aspect = read_annotations(filepath, filename)
    articles, reference_summaries = get_article_summaries(annotations)
    for art, ref in list(zip(articles, reference_summaries)):
        df_dict['id'].append(random.getrandbits(128))
        df_dict['Article'].append(art)
        df_dict['Aspect'].append(aspect)
        df_dict['Query'].append(query)
        df_dict['Reference'].append(ref)
        df_dict['Faithful_Query'].append('yes')
        
    assert(len(set([len(v) for k , v in df_dict.items()])) == 1)
    return df_dict
        


In [124]:
df_dict = {
    'id': [],
    'Article': [],
    'Aspect': [],
    'Query': [],
    'Reference': [],
    'Faithful_Query': []

}
filepath = '/work/frink/ramprasad.sa/aosumm/data'
filename = 'fraud_100_penalty.txt'
df_dict = populate_df(filepath, filename, df_dict)
len(df_dict['id'])

101

In [125]:
filepath = '/work/frink/ramprasad.sa/aosumm/data'
filename = 'fraud_100_nature.txt'
df_dict = populate_df(filepath, filename, df_dict)
len(df_dict['id'])

202

In [126]:
filepath = '/work/frink/ramprasad.sa/aosumm/data'
filename = 'eq_100_geo.txt'
df_dict = populate_df(filepath, filename, df_dict)
len(df_dict['id'])

303

In [127]:
filepath = '/work/frink/ramprasad.sa/aosumm/data'
filename = 'eq_100_geo.txt'
df_dict = populate_df(filepath, filename, df_dict)
len(df_dict['id'])

404

In [128]:
df = pd.DataFrame(df_dict)

In [129]:
df
        


,id,Article,Aspect,Query,Reference,Faithful_Query
0,247543995101515407153817589358894090994,Lawyers for jailed millionaire Robert Durst sa...,"penalty or consequences for the fraudster, or ...","penalty or consequences for the fraudster, or ...",Lawyers for jailed millionaire Robert Durst sa...,yes
1,333173560690393269554370932061037991079,Oregon-based defense contractor FLIR Systems I...,"penalty or consequences for the fraudster, or ...","penalty or consequences for the fraudster, or ...",Oregon-based defense contractor FLIR Systems I...,yes
2,236792177404515497775082764168118072571,Google Wallet says it has changed its policy w...,"penalty or consequences for the fraudster, or ...","penalty or consequences for the fraudster, or ...","For example, for those who use mobile payment ...",yes
3,262417639128301614252128265246116796974,"Tamara Pacskowska, who posed as a wealthy pens...","penalty or consequences for the fraudster, or ...","penalty or consequences for the fraudster, or ...",A Polish conwoman who posed as a wealthy pensi...,yes
4,99324562146308020801814252666146204731,"'The General' Mohammed Suleman Khan, 43, was ...","penalty or consequences for the fraudster, or ...","penalty or consequences for the fraudster, or ...","'The General' Mohammed Suleman Khan, 43, was ...",yes
...,...,...,...,...,...,...
399,96772121434302228524035801668221220614,(CNN) -- A strong 6.6-magnitude earthquake str...,"geography, region, or location","geography, region, or location",(CNN) -- A strong 6.6-magnitude earthquake str...,yes
400,62482699960013645282739550804782713032,(CNN) -- Confusion and then an uneasy calm.\nT...,"geography, region, or location","geography, region, or location",That's how residents up and down the East Coas...,yes
401,328844335995869382036570946417926490143,(CNN) -- Cricket officials plan to commemorate...,"geography, region, or location","geography, region, or location",Emergency workers are searching for survivors ...,yes
402,134294467637613055190980839641912579737,(CNN) -- An early morning earthquake rattled n...,"geography, region, or location","geography, region, or location",(CNN) -- An early morning earthquake rattled n...,yes


In [131]:
df.to_csv('/home/ramprasad.sa/qf_sd_summarization/datasets/postprocessed/aspect_focused/aspect_news_summ.csv')


